In [183]:
# import dependencies

import dash
from dash import Dash, html, dcc 

import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output
import random
goodreads_df = pd.read_csv('data.csv')

In [251]:
app = Dash(__name__)

server = app.server

genres = [
    'adult', 'art', 'audiobook', 'biography', 'business', 'childrens', 'classics', 'comics',
    'comedy', 'contemporary', 'culture', 'food_and_drink', 'comedy', 'crime', 'fantasy',
    'fiction', 'sexuality_and_gender', 'literature', 'graphic_Novels', 'historical', 'history',
    'horror', 'magna', 'memoir', 'music', 'mystery', 'nonfiction', 'paranormal', 'philosophy',
    'poetry', 'psychology', 'religion', 'romance', 'science', 'science_fiction', 'self_help',
    'suspense', 'spirituality', 'sports', 'thriller', 'travel', 'young_adult'
]

color_palette = [px.colors.qualitative.Plotly[random.randint(0, len(px.colors.qualitative.Plotly) - 1)] for _ in range(len(genres))]

# Assign a color from the palette to each genre
color_mapping = {genre: color_palette[i] for i, genre in enumerate(genres)}


# Calculate the sum of values for each genre column
genre_counts = goodreads_df.iloc[:, 9:].sum()

# Create a bar plot using Plotly Express
fig = px.bar(
    x=genre_counts.index,
    y=genre_counts.values,
    labels={'x': 'Genre', 'y': 'Number of Books'},
    color=genre_counts.index.map(color_mapping),
)






app.layout = html.Div([
   html.Nav(
        [
            html.Ul(
    [
        html.Li(html.P("Goodreads Top 10,000 Dashboard", style={"padding": "10px 20px", "margin": "0", "background-color": "#666666",
                                   "border-radius": "5px", "list-style-type": "none","color": "white", "text-decoration": "none", "font-family": "Arial"})),
        html.Li(html.A("Personal Books", href="#", style={"color": "white", "text-decoration": "none", "font-family": "Arial"}),
                style={"padding": "5px 10px", "margin": "0", "background-color": "#666666",
                       "border-radius": "5px", "list-style-type": "none"}),  # Add styling to Li elements
        html.Li(html.A("Public Books", href="#", style={"color": "white", "text-decoration": "none", "font-family": "Arial"}),
                style={"padding": "5px 10px", "margin": "0", "background-color": "#666666",
                       "border-radius": "5px", "list-style-type": "none"}),  # Add styling to Li elements
        html.Li(html.A("About this Dashboard", href="#", style={"color": "white", "text-decoration": "none", "font-family": "Arial"}),
                style={"padding": "5px 10px", "margin": "0", "background-color": "#666666",
                       "border-radius": "5px", "list-style-type": "none"}),  # Add styling to Li elements
    ],
    className="nav",
    style={"display": "flex", "justify-content": "space-between","padding": "5px", "margin": "0"}  # Align items with space between
)
        ],
        className="navbar",
        style={"background-color": "lightgray",
               "padding": "5px",
               "width": "99%",
               "margin-right": "0px",
               "margin-left": "10px",
               "box-sizing": "border-box",
               "border-radius": "15px"
               }
    ),
    html.Div(
        [
            html.Div(
                [html.Label("Col 1", style={"color": "white"})],
                style={
                    "background-color": "lightgray",
                    "width": "32.5%",
                    "display": "inline-block",
                    "padding": "5px",
                    "margin-left": "5px",
                    "margin-right": "10px",
                    "margin-bottom": "20px",
                    "min-width": "350px",
                    "height": "100vh",  # Set height to fill the entire viewport height
                    "box-sizing": "border-box", 
                    "border-radius": "15px" # Include padding in height calculation
                },
            ),


            #######################################################################
            ###########################COLUMN 2 DROPDOWN GRAPH#####################
            #######################################################################
             html.Div(
                 [html.Div([
                    html.H3("Genre versus Quantity of Books", style={"text-align": "center","padding": "10px 20px", "margin": "0", "background-color": "#666666",
                                   "border-radius": "5px", "list-style-type": "none","color": "white", "text-decoration": "none", "font-family": "Arial"}),
                ],

                style={"margin-bottom": "20px"}
                 ),
                 html.Div([
                    html.H5("Select genres using the dropdown to populate the graph!", style={"text-align": "center", "margin": "0",
                                    "list-style-type": "none","color": "black", "text-decoration": "none", "font-family": "Arial"}),
                ],

                style={"margin-bottom": "20px"}
                 ),
        html.Div(
            [dcc.Dropdown(
                id='genre-dropdown',
                options=[{'label': genre.capitalize().replace('_', ' '), 'value': genre} for genre in genres],
                value=[],
                multi=True,
                persistence=True,
                style={'width': '100%',
                       }
            )],
            style={'width': '100%', 'margin': 'auto', 'margin-bottom': '20px','maxHeight': '300px',  } 
                
        ),
        # Graph for displaying number of books in selected genres
        dcc.Graph(
        id='genre-graph',
        figure=fig
    ), 
    html.Table(
                id='top-books-table',
                style={"width": "100%", "margin": "auto"}
            )
    ],
    style={
        "background-color": "lightgray",
        "width": "32.5%",
        "display": "inline-block",
        "border-right": "2px solid #333",
        "padding": "10px",
        "margin-left": "5px",
        "margin-right": "10px",
        "margin-bottom": "20px",
        "min-width": "350px",
        "height": "100vh",  # Set height to fill the entire viewport height
        "box-sizing": "border-box",
          "border-radius": "15px"  # Include padding in height calculation
    },
),
            html.Div(
                [html.Label("Col 3", style={"color": "white"})],
                style={
                    "background-color": "lightgray",
                    "width": "32.5%",
                    "display": "inline-block",
                    "padding": "5px",
                    "margin-bottom": "20px",
                    "min-width": "350px",
                    
                    "height": "100vh",  # Set height to fill the entire viewport height
                    "box-sizing": "border-box",
                      "border-radius": "15px"  # Include padding in height calculation
                },
            ),
        ],
        style={"margin": "10px auto", "max-width": "4000px", "width": "100%","text-align": "center","display": "flex"}  # Set width to 100% of parent container
    ),
],
style={"background-color": "#444444", "padding": "15px"}
)
###################################################################
########################CALLBACKS##################################
###################################################################
@app.callback(
    Output('genre-graph', 'figure'),
    [Input('genre-dropdown', 'value')]
)
@app.callback(
    Output('top-books-table', 'children'),
    [Input('genre-dropdown', 'value')]
)

###################################################################
###########################FUNCTIONS###############################
###################################################################

def update_genre_graph(selected_genres):
    if not selected_genres:
        # If no genre is selected, return an empty graph
        return {
            'data': [],
            'layout': {
                'title': 'Number of Books in Selected Genres',
                'xaxis': {'title': 'Genre'},
                'yaxis': {'title': 'Number of Books'},
                'showlegend': False,
                'paper_bgcolor': 'rgba(255, 255, 255, 0)',  # Make the background transparent
                'plot_bgcolor': 'rgba(255, 255, 255, 0)',   # Make the plot area background transparent
                'xaxis_showgrid': False,                    # Hide x-axis gridlines
                'yaxis_showgrid': False,                    # Hide y-axis gridlines
            }
        }
    
    # Calculate the sum of values for each selected genre column
    genre_counts = goodreads_df[selected_genres].sum()
    
    # Create bar traces for the selected genres with assigned colors
    data = []
    for genre in selected_genres:
        trace = {
            'x': [genre],
            'y': [genre_counts[genre]],
            'type': 'bar',
            'marker': {'color': color_mapping[genre]},  # Assign color based on the genre
            'text': [genre_counts[genre]],             # Set text to y-value of the bar
            'textposition': 'auto',                     # Automatically position the text above the bars
        }
        data.append(trace)
    
    return {
        'data': data,
        'layout': {
            'xaxis': {'title': 'Genre'},
            'yaxis': {'title': 'Number of Books'},
            'showlegend': False,
            'paper_bgcolor': 'rgba(255, 255, 255, 0)',  # Make the background transparent
            'plot_bgcolor': 'rgba(255, 255, 255, 0)',   # Make the plot area background transparent
            'xaxis_showgrid': False,                    # Hide x-axis gridlines
            'yaxis_showgrid': False,                    # Hide y-axis gridlines
        }
    }

def update_top_books_table(selected_genres):
    if not selected_genres:
        return ''  # No selected genres, return an empty string
    
    # Filter the DataFrame to include only the selected genres
    filtered_df = goodreads_df[goodreads_df['Genre'].isin(selected_genres)]
    
    # Sort the DataFrame by number of reviews in descending order
    sorted_df = filtered_df.sort_values(by='Num Reviews', ascending=False)
    
    # Take the top 5 books
    top_books = sorted_df.head(5)
    
    # Create table rows
    table_rows = [
        html.Tr([
            html.Th("Book Title"),
            html.Th("Num Reviews"),
            html.Th("Num Written Reviews")
        ]),  # Table header
    ]
    for _, row in top_books.iterrows():
        table_rows.append(
            html.Tr([
                html.Td(row['Book Title']),
                html.Td(row['Num Reviews']),
                html.Td(row['Num Written Reviews'])
            ])
        )
    
    return html.Table(table_rows)

if __name__ == "__main__":
    app.run(jupyter_mode = 'tab', debug=False)


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>